### 24.03.13. 과제
- mnist fashion 자료 가져와서 분류 해보기
- 와인 자료 가져와서 회귀와 분류 해보기

In [69]:
from sklearn.datasets import fetch_openml
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

In [70]:
file = fetch_openml('Fashion-MNIST', parser = 'auto')
data = file['data']
data

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,...,119,114,130,76,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,22,...,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,33,96,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
69996,0,0,0,0,0,0,0,0,0,31,...,0,0,0,0,0,0,0,0,0,0
69997,0,0,0,0,0,0,0,0,0,0,...,27,0,0,0,0,0,0,0,0,0
69998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [71]:
target = file['target']
target.unique()

['9', '0', '3', '2', '7', '5', '1', '6', '4', '8']
Categories (10, object): ['0', '1', '2', '3', ..., '6', '7', '8', '9']

In [72]:
target = target.astype('int64')
target.unique()

array([9, 0, 3, 2, 7, 5, 1, 6, 4, 8], dtype=int64)

In [73]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, target, stratify = target, random_state = 42)

In [74]:
print(f"[훈련용] X_train : {X_train.shape}     y_train : {y_train.shape}")
print(f"[테스트용] X_test : {X_test.shape}      y_test : {y_test.shape}")

[훈련용] X_train : (52500, 784)     y_train : (52500,)
[테스트용] X_test : (17500, 784)      y_test : (17500,)


In [75]:
# 랜덤 시드 설정
torch.manual_seed(1)

In [82]:
# 훈련 데이터
X_train = torch.Tensor(np.array(X_train))
X_test = torch.Tensor(np.array(X_test))
y_train = torch.IntTensor(np.array(y_train))
y_test = torch.IntTensor(np.array(y_test))

In [83]:
# 모델 설계
# sequential : 순서대로 진행되어야 하는 모듈을 묶음으로 관리

model = nn.Sequential(
    nn.Linear(784, 1),        # input = 2, output = 1
    nn.Sigmoid()
)

optimizer = optim.SGD(model.parameters(), lr = 1)

In [84]:
# 학습 

def training():
    nb_epochs = 1000
    for epoch in range(nb_epochs + 1):
        # 예측값 계산
        hypothesis = model(X_train)
        
        # cost 계산 => 손실함수 cross_entropy()
        cost = F.cross_entropy(hypothesis, y_train)
        
        # cost로 H(x) 계산
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        # 정확도 검사        
        prediction = hypothesis >= torch.FloatTensor([0.5])
        correct_prediction = prediction.float() == y_train
        accuracy = correct_prediction.sum().item() / len(correct_prediction)
        
        print(f'Epoch {epoch:4d}/{nb_epochs} Cost: {cost.item():.6f}')


In [85]:
training()

RuntimeError: expected scalar type Long but found Int